<a href="https://colab.research.google.com/github/furro33/College-football-development-/blob/main/cfb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:

drive.mount('/content/gdrive')

Mounted at /content/gdrive


GetDraftClassData extract and make some transformations draft class data from pro-football-reference

In [ ]:
def GetDraftClassData(link):
    url = link
    info = pd.read_html(url)[0]
    df = pd.DataFrame(info)
    pd.set_option('max_columns', 50)
    df.columns = [col[1] for col in info.columns]
    df.drop(info.iloc[:, 8:25], inplace=True, axis=1)
    df.drop(['AP1', 'Sk'], axis=1, inplace=True)
    df.drop(['Unnamed: 28_level_1'], axis=1, inplace=True)
    #print(df.dtypes)
    df = df.drop(df[df.Age == "Age"].index) 
    df["Year"] = 2000
    df['Player'] = df['Player'].str.replace("HOF", ' ') #reaplce "HOF" with " "
    df.rename(columns={"Rnd" : "Round",
                                 "Tm": "Team",
                                 "Pos": "Position",
                                 "To":"RetYear",
                                 "College/Univ": "College",
                                 "Year": "DraftClass"}, inplace=True)
    #df[['Age','RetYear']].astype(int)
    #df.fillna(df.mean(), inplace = True)
    #df.to_csv("scooby.csv")
    return df

Working with NaN on Age and RetYear

In [ ]:
def RemoveNans(link):
  df = GetDraftClassData(link)  
  df[['Age','RetYear']] = df[['Age','RetYear']].fillna(0)  #fill NaNs with 0
  df["Age"] = pd.to_numeric(df["Age"]) #convert object tu numeric value
  df["RetYear"] = pd.to_numeric(df["RetYear"])
  df["Round"] = pd.to_numeric(df["Round"])
  df["Pick"] = pd.to_numeric(df["Pick"])
  df['Age']=df['Age'].replace(0,method='ffill') # replace 0 with previous value 
  df['RetYear']=df['RetYear'].replace(0, method='ffill')
  return df


  #return df.dtypes


Extract clean data from draft classes (2000-2020)

In [ ]:
def draftclasses():
  for i in range(21):
      if i < 10:
          #print("https://www.pro-football-reference.com/years/200" + str(i) + "/draft.htm")
          file = RemoveNans(f"https://www.pro-football-reference.com/years/200{i}/draft.htm")
          file["DraftClass"] = (f"200{i}")
          name = f"Draft 200{i}"
          file.to_csv(f"{name}.csv")
          #df.to_csv(f'{i}.csv')
      else:
          #print("https://www.pro-football-reference.com/years/20" + str(i) + "/draft.htm")
          file = RemoveNans("https://www.pro-football-reference.com/years/20" + str(i) + "/draft.htm")
          file["DraftClass"] = (f"20{i}")
          name = f"Draft 20{i}"
          file.to_csv(f"{name}.csv")

  #/gdrive/MyDrive/cfb/Draft/


In [ ]:
draftclasses()